##  Transfer Learning
- 예를 들어 강아지와 고양이를 구분하는 딥러닝 모델을 구축하고자 한다.
- 데이터가 100장밖에 없다면 미리 학습해 놓은 Pre_trained model을 가져와 fine tuning 을 시켜야한다.
> 이러한 방법을 Transfer Learning 이라고 한다. == 전이학습

### (1) 일반적인 방법론
- 보통 학습되어 있는 pretrained model은 훨씬 큰 문제를 푸는 모델이기 때문에 output layer를 수정해야 한다.
- 그리고 우리가 보유한 데이터를 input으로 학습을 진행해야한다.
- 일반적으로 pre trained model 의 fc layer는 이전의 weight 는 학습시키지 않는다. == weight freezing이라고 표현
- 다시말해 fc layer 이전에 weight 들은 freezing을 시켜놓고 우리의 데이터를 통해 fc layer 의 weight 만 학습을 시킨다는 말이다.
- 이 과정을 fine tuning 이라고 한다.
- Transfer Learning은 Initialization 의 개념으로도 볼 수 있다. parameter 들의 초기값을 우리는 특정 기법을 쓰는 것이 아닌 Pre_trained model의 가중치들을 사용
- 일반적으로 데이터가 많지 않은 상황에서 보유한 데이터만으로 학습시킨 것보다 전이학습을 한 경우에서 모델 성능이 높고, 학습을 더 빠르게 시킬 수 있다는 장점

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [3]:
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'
BATCH_SIZE = 32
EPOCHS = 20

print('torch version : {}'.format(torch.__version__))
print('torch cuda Device : {}'.format(DEVICE))
print('batch_size : {}\nEpochs : {}'.format(BATCH_SIZE,EPOCHS))

torch version : 1.7.1+cu110
torch cuda Device : cuda
batch_size : 32
Epochs : 20


In [14]:
data_transforms = {
    'train' : transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ]),
    'val' : transforms.Compose([
        transforms.CenterCrop(224),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
}

train_datasets = datasets.ImageFolder("./data/hymenoptera_data",transforms = data_transforms['train'])
test_datasets = datasets.ImageFolder("./data/hymenoptera_data",transforms = data_transforms['val'])
image_datasets = {x : datasets.ImageFolder("./data/hymenoptera_data",data_transforms[x]) for x in ['train','val']}
dataloader = {x : torch.utils.data.DataLoader(image_datasets[x], batch_size = BATCH_SIZE, num_workers = 0, shuffle = True) for x in ['train','val']}

In [15]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 397
     Root location: ./data/hymenoptera_data
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            ),
 'val': Dataset ImageFolder
     Number of datapoints: 397
     Root location: ./data/hymenoptera_data
     StandardTransform
 Transform: Compose(
                CenterCrop(size=(224, 224))
                Resize(size=256, interpolation=PIL.Image.BILINEAR)
                ToTensor()
                Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            )}

In [21]:
for a,b in dataloader['train']:
    print(a.size())
    print(b.size())
    break

torch.Size([32, 3, 224, 224])
torch.Size([32])
